# NFL Big Data Bowl 2019 EDA
 
 This will be my beginning EDA to expore the data.
 This is my basic eploritory before looking at other EDA's code

## Setup
 Imports and getting the data into a dataframe. Basic exploritory of it

In [3]:
# Imports
import os
import datetime
# from kaggle.competitions import nflrush
import tqdm

import pandas as pd
import numpy as np
import math
from scipy.spatial import distance
import matplotlib.pyplot as plt
import matplotlib.patches as patches

import tensorflow as tf
from tensorflow.keras import Sequential, layers, Model
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, LeakyReLU, Input, PReLU, Add
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [6]:
for dirname, _, filenames in os.walk('./kaggle'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

./kaggle/competitions/nflrush/sample_submission.csv.encrypted
./kaggle/competitions/nflrush/__init__.py
./kaggle/competitions/nflrush/competition.cpython-36m-x86_64-linux-gnu.so
./kaggle/competitions/nflrush/test.csv.encrypted
./kaggle/competitions/nflrush/__pycache__/__init__.cpython-37.pyc


In [ ]:
train_df = pd.read_csv('../input/nfl-big-data-bowl-2020/train.csv', low_memory=False)
train_df.tail()

In [ ]:
train_df.info()

In [ ]:
avg_dis= train_df.query('NflIdRusher == NflId').groupby('DisplayName')[['Dis','DisplayName']].mean()
avg_dis.sort_values(by=['Dis'])

In [ ]:
train_df['GameId'].value_counts()

## Displaying of players
 Using the code from this notebook. It is a simple solution to displaying the players. 
 And the time spent making it can be put to better use
 https://www.kaggle.com/robikscube/nfl-big-data-bowl-plotting-player-position

In [ ]:
pd.set_option('max_columns', 100)

train = pd.read_csv('../input/nfl-big-data-bowl-2020/train.csv', low_memory=False)

def create_football_field(linenumbers=True,
                          endzones=True,
                          highlight_line=False,
                          highlight_line_number=50,
                          highlighted_name='Line of Scrimmage',
                          fifty_is_los=False,
                          figsize=(12, 6.33)):
    """
    Function that plots the football field for viewing plays.
    Allows for showing or hiding endzones.
    """
    rect = patches.Rectangle((0, 0), 120, 53.3, linewidth=0.1,
                             edgecolor='r', facecolor='darkgreen', zorder=0)

    fig, ax = plt.subplots(1, figsize=figsize)
    ax.add_patch(rect)

    plt.plot([10, 10, 10, 20, 20, 30, 30, 40, 40, 50, 50, 60, 60, 70, 70, 80,
              80, 90, 90, 100, 100, 110, 110, 120, 0, 0, 120, 120],
             [0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3,
              53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 53.3, 0, 0, 53.3],
             color='white')
    if fifty_is_los:
        plt.plot([60, 60], [0, 53.3], color='gold')
        plt.text(62, 50, '<- Player Yardline at Snap', color='gold')
    # Endzones
    if endzones:
        ez1 = patches.Rectangle((0, 0), 10, 53.3,
                                linewidth=0.1,
                                edgecolor='r',
                                facecolor='blue',
                                alpha=0.2,
                                zorder=0)
        ez2 = patches.Rectangle((110, 0), 120, 53.3,
                                linewidth=0.1,
                                edgecolor='r',
                                facecolor='blue',
                                alpha=0.2,
                                zorder=0)
        ax.add_patch(ez1)
        ax.add_patch(ez2)
    plt.xlim(0, 120)
    plt.ylim(-5, 58.3)
    plt.axis('off')
    if linenumbers:
        for x in range(20, 110, 10):
            numb = x
            if x > 50:
                numb = 120 - x
            plt.text(x, 5, str(numb - 10),
                     horizontalalignment='center',
                     fontsize=20,  # fontname='Arial',
                     color='white')
            plt.text(x - 0.95, 53.3 - 5, str(numb - 10),
                     horizontalalignment='center',
                     fontsize=20,  # fontname='Arial',
                     color='white', rotation=180)
    if endzones:
        hash_range = range(11, 110)
    else:
        hash_range = range(1, 120)

    for x in hash_range:
        ax.plot([x, x], [0.4, 0.7], color='white')
        ax.plot([x, x], [53.0, 52.5], color='white')
        ax.plot([x, x], [22.91, 23.57], color='white')
        ax.plot([x, x], [29.73, 30.39], color='white')

    if highlight_line:
        hl = highlight_line_number + 10
        plt.plot([hl, hl], [0, 53.3], color='yellow')
        plt.text(hl + 2, 50, '<- {}'.format(highlighted_name),
                 color='yellow')
    return fig, ax

def plot_runner(runner_df, ax):
    x = runner_df['X'].values[0]
    y = runner_df['Y'].values[0]
    rad = math.radians(runner_df['Dir'].values[0])
    dx = math.cos(rad) * 5
    dy = math.sin(rad) * 5
    print(runner_df['Dir'].values[0])
    ax.plot(x=x, y=y, kind='scatter', color='red', s=30, legend='Runner')
    ax.arrow(x=x, y=y, dx=dx, dy=dy, head_width=1)

def display_play(playid):
    yl = train.query("PlayId == @playid")['YardLine'].tolist()[1]
    fig, ax = create_football_field(highlight_line=True,
                                    highlight_line_number=yl+54)
    plot_runner(train.query("PlayId == @playid and NflIdRusher == NflId"), ax)
    train.query("PlayId == @playid and Team == 'away'") \
        .plot(x='X', y='Y', kind='scatter', ax=ax, color='orange', s=30, legend='Away')
    train.query("PlayId == @playid and Team == 'home' and NflIdRusher != NflId") \
        .plot(x='X', y='Y', kind='scatter', ax=ax, color='blue', s=30, legend='Home')
    plt.title(f'Play # {playid}')
    plt.legend()

    
playid = 20181230154157
display_play(playid)
plt.show()

In [ ]:
game_df = train_df.query('GameId == 2018123015')
i = 0
total_plays = 0
for playid in game_df.PlayId.unique():
    display_play(playid)
    i += 1
    if i > total_plays:
        break

plt.show()

## Submission data
 I'll go through the test data and example submission

In [ ]:
submit = False
if submit:
    env = nflrush.make_env()

    for (test_df, sample_prediction_df) in env.iter_test():
        env.predict(sample_prediction_df)
    env.write_submission_file()

# Normalise the data
## Same direction
 First step is to have the attacking always from left to right

In [ ]:
def correct_team_abbr(df_input):
    df = df_input.copy()
    map_abbr = {'ARI': 'ARZ', 'BAL': 'BLT', 'CLE': 'CLV', 'HOU': 'HST'}
    for abb in train['PossessionTeam'].unique():
        map_abbr[abb] = abb
    df['PossessionTeam'] = df['PossessionTeam'].map(map_abbr)
    df['HomeTeamAbbr'] = df['HomeTeamAbbr'].map(map_abbr)
    df['VisitorTeamAbbr'] =df['VisitorTeamAbbr'].map(map_abbr)
    return df

def correct_direction(df):
    std_df = df.copy()
    plays_incor_dir = train_df.query("NflIdRusher == NflId  and PlayDirection == 'left'")['PlayId'].values
    home_has_the_ball = train_df.query("NflIdRusher == NflId and Team == 'home'")['PlayId'].values
    away_has_the_ball = train_df.query("NflIdRusher == NflId and Team == 'away'")['PlayId'].values
    std_df['X_Std'] = np.where(std_df['PlayId'].isin(plays_incor_dir), 120-std_df['X'], std_df['X'])
    std_df['Y_Std'] = np.where(std_df['PlayId'].isin(plays_incor_dir), 53.3-std_df['Y'], std_df['Y'])
    std_df['YardLine_Std'] = np.where(std_df['PlayId'].isin(plays_incor_dir), 100-std_df['YardLine'], std_df['YardLine'])
    std_df['Orientation_Std'] = np.where(std_df['PlayId'].isin(plays_incor_dir), (std_df['Orientation']+180) % 360, std_df['Orientation'])
    std_df['Dir_Std'] = np.where(std_df['PlayId'].isin(plays_incor_dir), (std_df['Dir']+180) % 360, std_df['Dir'])
    std_df['HasBall'] = np.where(np.logical_or(np.logical_and(std_df['PlayId'].isin(home_has_the_ball), std_df['Team'] == 'home'), np.logical_and(std_df['PlayId'].isin(away_has_the_ball), std_df['Team'] == 'away')), True, False)
    std_df['IsRusher'] = np.where(std_df['NflIdRusher'] == std_df['NflId'], True, False)
    return std_df

def fix_data(df_input):
    df = correct_team_abbr(df_input)
    df = correct_direction(df)
    return df
    

def handle_na(df):
    return df.dropna()

# std_df = correct_direction(train_df)
# std_df.head()

## Column Fixing
 Columns can be replaced with other values that will allow the deep learning algorithm to work correctly


In [ ]:
def strtoseconds(txt):
    txt = txt.split(':')
    ans = int(txt[0])*60 + int(txt[1]) + int(txt[2])/60
    return ans

def normalise_windspeed(input_df):
    df = input_df.copy()
    df['WindSpeed'] = input_df.query("WindSpeed in ('E', 'SE', 'SSW')")['WindDirection']
    df['WindDirection'] = input_df.query("WindSpeed in ('E', 'SE', 'SSW')")['WindSpeed']
    df['WindSpeed'] = df["WindSpeed"].apply(lambda x: 0 if x == 'Calm' else x)
    df['WindSpeed'] = df['WindSpeed'].apply(lambda x: x.lower().replace('mph', '').strip() if not pd.isna(x) else x)
    df['WindSpeed'] = df['WindSpeed'].apply(lambda x: (int(x.split('-')[0])+int(x.split('-')[1]))/2 if not pd.isna(x) and '-' in x else x)
    df['WindSpeed'] = df['WindSpeed'].apply(lambda x: (int(x.split()[0])+int(x.split()[-1]))/2 if not pd.isna(x) and type(x)!=float and 'gusts up to' in x else x)
    df['WindSpeed'] = df['WindSpeed'].astype(float)
    return df
    
def normalise_weather(input_df):
    return input_df.drop(['GameWeather'], axis=1)
    
def normalise_columns(input_df):
    df = input_df.copy()
    df['PlayerHeight'] = df['PlayerHeight'].apply(lambda x: 12*int(x.split('-')[0])+int(x.split('-')[1]))
    df['GameClock'] = df['GameClock'].apply(strtoseconds)
    df = pd.concat([df.drop(['OffenseFormation'], axis=1), pd.get_dummies(df['OffenseFormation'], prefix='Formation')], axis=1)
    df['TimeHandoff'] = df['TimeHandoff'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))
    df['TimeSnap'] = df['TimeSnap'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))
    df['TimeDelta'] = df.apply(lambda row: (row['TimeHandoff'] - row['TimeSnap']).total_seconds(), axis=1)
    df['PlayerBirthDate'] = df['PlayerBirthDate'].apply(lambda x: datetime.datetime.strptime(x, "%m/%d/%Y"))
    df['PlayerAge'] = df.apply(lambda row: (row['TimeHandoff']-row['PlayerBirthDate']).total_seconds()/31557600.0, axis=1)
    df = df.drop(['TimeHandoff', 'TimeSnap', 'PlayerBirthDate'], axis=1)
    df['Team'] = df['Team'].apply(lambda x: x.strip()=='home')
    df = normalise_windspeed(df)
    df = normalise_weather(df)
    return df

# std_df = normalise_columns(std_df)
# std_df.head()

## Rusher related stats

In [ ]:
def add_defensive_stats(train_df_input, df_input):
    df = df_input.copy()
    train_df = train_df_input.copy()
    train_df['DefensiveTeam'] = np.where(train_df['PossessionTeam'] == train_df['HomeTeamAbbr'], train_df['VisitorTeamAbbr'], train_df['HomeTeamAbbr'])
    mean_dict = train_df.query('IsRusher == True').groupby('DefensiveTeam')['Yards'].mean().to_dict()
    df['DefensiveYardsAgainst'] = train_df['DefensiveTeam'].map(mean_dict)
    return df
    
def add_season_mean(train_df,df_input):
    df = df_input.copy()
    mean_dict = train_df.query("IsRusher == True").groupby(['Season','DisplayName'])['Yards'].mean()
    for year in range(2017, 2020):
        try:
            df["RusherMeanYards_{}".format(year)] = df['DisplayName'].map(mean_dict[year])
        except IndexError:
            pass
    return df

def add_rusher_stats(train_df,df_input):
    df = df_input.copy()
    mean_dict = train_df.query('IsRusher == True').groupby('DisplayName')['Yards'].mean().to_dict()
    df['RusherMeanYards'] = df['DisplayName'].map(mean_dict)
    return df

def add_yards_stats(train_df,df_input):
    df = df_input.copy()
    df = add_rusher_stats(train_df,df)
    df = add_season_mean(train_df,df)
    df = add_defensive_stats(train_df,df)
    df = df.fillna(-1)
    return df
    
#add_yards_stats(std_df_nor, std_df_nor.iloc[0:22]).query('IsRusher == True').head()

## Closest Defense
 Getting the closest defensive player to the runner

In [ ]:
def add_closest_defensive_player(df):
    playids = df.query("NflIdRusher == NflId")['PlayId'].values
    df['ClosestDefense'] = -1.0
    for playid in playids:
        runner = df.query("PlayId == @playid and IsRusher == True")
        defense = df.query("PlayId == @playid and HasBall == False")
        d_pos = defense[['X_Std', 'Y_Std']].values
        r_pos = runner[['X_Std', 'Y_Std']].values
        min_dis = distance.cdist(r_pos,d_pos).min(axis=1)
        df.at[runner.index.values.astype(int)[0], 'ClosestDefense'] = min_dis
    return df

#Want to get closest player in FoV
def add_closest_fov_player(df):
    playids = df.query("NflIdRusher == NflId")['PlayId'].values
    df['ClosestDefense'] = -1.0
    df['ClosestDefenseFov'] = -1.0
    for playid in playids:
        runner = df.query("PlayId == @playid and IsRusher == True")
        defense = df.query("PlayId == @playid and HasBall == False")
    
        d_pos = defense[['X_Std', 'Y_Std']].values
        r_pos = runner[['X_Std', 'Y_Std']].values
        min_dis = distance.cdist(r_pos,d_pos).min(axis=1)
        df.at[runner.index.values.astype(int)[0], 'ClosestDefense'] = min_dis
        
        if ((runner['Orientation_Std'] >= 45) | (runner['Orientation_Std'] <= 135)).values[0]:
            y_max = runner['Y_Std'].values[0]-3
            defense = defense[defense['Y_Std']>y_max]
        elif ((runner['Orientation_Std'] >= 225) | (runner['Orientation_Std'] <= 305)).values[0]:
            y_max = runner['Y_Std'].values[0]+3
            defense = defense[defense['Y_Std']<y_max]
        d_pos = defense[['X_Std', 'Y_Std']].values
        r_pos = runner[['X_Std', 'Y_Std']].values
        try:
            min_dis = distance.cdist(r_pos,d_pos).min(axis=1)
        except ValueError:
            min_dis = -1
        df.at[runner.index.values.astype(int)[0], 'ClosestDefenseFov'] = min_dis
    return df

def closest_player_stats(df_input):
    df = df_input.copy()
    df = add_closest_fov_player(df)
    #df = add_closest_defensive_player(df)
    return df

## Flatten Data
 Get all players data into a single row

In [ ]:
# Remove some columns at the moment
def remove_columns(df):
    cat_features = []
    for col in df.columns:
        if df[col].dtype =='object':
            cat_features.append(col)
    df = df.drop(cat_features, axis=1)
    return df

def flatten(df_input):
    df = df_input.copy()
    df = df.sort_values(by=['PlayId', 'HasBall', 'IsRusher']).reset_index()
    player_columns = ['JerseyNumber','PlayerHeight','PlayerWeight','PlayerAge','X_Std','Y_Std','Orientation_Std','Dir_Std','S','A']
    drop_columns = ['X','Y','Dis','Orientation','HasBall','IsRusher', 'Team','NflId','DisplayName','PlayerCollegeName','FieldPosition','StadiumType']
    
    player_np = np.array(df[player_columns].fillna(df[player_columns].mean())).reshape(-1, 10*22)
    play_df = df.drop(columns=player_columns, axis=1).query('IsRusher == True').drop(drop_columns, axis=1)
    play_df = remove_columns(play_df)
    play_df = play_df.fillna(play_df.mean())
    
    play_np = np.array(play_df)
    X = np.concatenate((play_np, player_np), axis=1)
    return play_np

#flatten(std_df)

# Saving dataframe and reloading it
 My dataframe is slow to build. Let's save it to disk

In [ ]:
try:
    std_df = pd.read_pickle("/kaggle/input/nflrunnerdataframe/std_df.pkl")
    print('Created Dataframe')
except FileNotFoundError:
    print('Not Found')

In [ ]:
try:
    std_df
except (AttributeError, NameError):
    std_df_c_d = fix_data(train_df)
    std_df_nor = normalise_columns(std_df_c_d)
    std_with_yards = add_yards_stats(std_df_nor, std_df_nor)
    std_df = closest_player_stats(std_with_yards)
    std_df.to_pickle("std_df.pkl")

# Making a model
 Given our data how would we go about making a small model to make the prediction
 
 Y values should be organised of 0.0 for yards it got past. 
 And 1.0 for yards it reached. An example of a player that reached 2 yards
     
     yards  -99  -98  -97  -96  ....  -1  0  1  2  3  4  .... 97  98  99
     result   0    0    0    0         0  0  0  1  1  1        1   1   1

In [ ]:
# Flatten data and get single one yards amount per row
Y_set = std_df.query('NflIdRusher == NflId')['Yards']
X_set = flatten(std_df)

# Split into train and test
X_train, X_test, y_train_yards, y_test_yards = train_test_split(X_set, Y_set, test_size=0.1, random_state=1)
X_train = scaler.fit_transform(X_train) 
X_test = scaler.transform(X_test) 


Y_train=np.zeros((X_train.shape[0],199))
for i,yard in enumerate(y_train_yards):
    Y_train[i, yard+99:] = np.ones(shape=(1, 100-yard))

Y_test=np.zeros((X_test.shape[0],199))
for i,yard in enumerate(y_test_yards):#
    Y_test[i, yard+99:] = np.ones(shape=(1, 100-yard))

In [ ]:
def crps(y_true,y_pred):
    diffs = (y_true - y_pred) ** 2
    sum_tf = tf.math.reduce_sum(tf.math.reduce_sum(diffs, axis=1), axis=0)
    return sum_tf / (199 * 32)

In [ ]:
# class OuputLayer(layers.Layer):

#     def __init__(self, **kwargs):
#         super(OuputLayer, self).__init__(**kwargs)
    
#     def get_output_shape_for(self, input_shape):
#         return (input_shape[0], self.n_mels, input_shape[2])
    
#     def call(self, inputs):
#         yardline = tf.Variable(inputs[0][0]).read_value()
#         print(yardline)
#         dense_output = inputs[1]
#         zeros = tf.Variable(tf.zeros(shape=(199,),dtype = tf.float32))
#         ones = tf.Variable(tf.ones(shape=(199,),dtype = tf.float32))
# #         tf.assign(dense_output[0:99-yardline], zeros[0:99-yardline])
# #         tf.assign(dense_output[99+yardline:-1], ones[99+yardline:-1])
#         return dense_output

def create_model(input_size):
    inputs = Input(input_size)
#     yardline_input = Input(1)
    x = Dense(input_size, activation='relu')(inputs)
    x = Dense(input_size, activation='relu')(inputs)
#     x = BatchNormalization()(x)
#     x = Dense(512, activation=None)(x)
#     x = PReLU()(x)
#     x = Dropout(0.5)(x)
#     x = Dense(512, activation=None)(x)
#     x = PReLU()(x)
#     shortcut = x
#     x = Dropout(0.5)(x)
#     x = Dense(512, activation=None)(x)
#     x = PReLU()(x)
#     x = Add()([x, shortcut])
#     x = Dropout(0.5)(x)
    x = Dense(199, activation='sigmoid')(x)
#     classifier = OuputLayer()([yardline_input,x])
#     model = Model(inputs=[inputs,yardline_input], outputs=[classifier])
    model = Model(inputs=[inputs], outputs=[x])
    model.compile(optimizer=Adam(learning_rate=0.00005), loss=['mse'], metrics=[crps])
    return model
model = create_model(X_train.shape[1])

In [ ]:
# def create_model(input_shape):
#     model = Sequential()
#     model.add(Dense(256, input_shape=[input_shape], activation='relu'))
#     model.add(BatchNormalization())
#     model.add(LeakyReLU(0.3))
#     model.add(Dropout(0.5))
#     model.add(Dense(256, activation='relu'))
#     model.add(BatchNormalization())
#     model.add(LeakyReLU(0.3))
#     model.add(Dropout(0.5))
#     model.add(Dense(256, activation='relu'))
#     model.add(BatchNormalization())
#     model.add(LeakyReLU(0.2))
#     model.add(Dropout(0.4))
#     model.add(Dense(199, activation='sigmoid'))

#     model.compile(optimizer=Adam(learning_rate=0.00005), loss=['mse'], metrics=[crps])
#     return model
# model = create_model(X_train.shape[1])

In [ ]:
checkpoint_path = "model.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_best_only=True,
                                                 verbose=1)
history = model.fit(X_train, 
                  Y_train, 
                  epochs=10,
                  batch_size=32,
#                   callbacks=[cp_callback],  
                  validation_data=(X_test, Y_test))

# model.load_weights("model.ckpt")

In [ ]:
def make_pred(df, model, train_df, env=None, sample=None):
    test_df = fix_data(df)
    test_df = normalise_columns(test_df)
    test_df = add_yards_stats(train_df, test_df)
    test_df = closest_player_stats(test_df)
    
    dummy_col = train_df.columns
    missing_cols = set( dummy_col ) - set( test_df.columns )-set('Yards')
    for c in missing_cols:
        test_df[c] = 0
    test_df = test_df.drop('Yards',axis=1)
    x_pred = flatten(test_df)
    x_pred = scaler.transform(x_pred)
    y_pred = np.zeros((1,199))
    y_pred = model.predict(x_pred)
    for pred in y_pred:
        pred[0:99-test_df['YardLine_Std'].values[0]] = 0.0
        pred[199-test_df['YardLine_Std'].values[0]:-1] = 1.0
        prev = 0
        for i in range(len(pred)):
            if pred[i]<prev:
                pred[i]=prev
            prev=pred[i]
    if (env):
        env.predict(pd.DataFrame(data=y_pred,columns=sample.columns))
    return y_pred
    
for i in range(0,5):
    fig, ax = plt.subplots(1)
    a = make_pred(train_df.iloc[i*22:22+(i*22)], model, std_df)
    # a = make_pred(stupid[0], model, std_df)
    plt.plot(np.arange(-99,100), a[0])
    plt.show()

In [0]:
stupid = []

In [0]:
env = nflrush.make_env()
for (test_df, sample_prediction_df) in env.iter_test():
    try:
        make_pred(test_df, model, std_df, env=env, sample=sample_prediction_df)
    except ValueError:
        stupid.append(test_df)
env.write_submission_file()